<a href="https://colab.research.google.com/github/Dahan15/-spam-text-message-classification/blob/main/spam_text_message_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.
import kagglehub
organizations_team_ai_spam_text_message_classification_path = kagglehub.dataset_download('organizations/team-ai/spam-text-message-classification')

print('Data source import complete.')


In [ ]:
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))


/kaggle/input/spam-text-message-classification/SPAM text message 20170820 - Data.csv


In [ ]:
import pandas as pd
import tensorflow_datasets as tfds
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
import tensorflow_hub as hub
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, SimpleRNN,Conv1D, GlobalMaxPooling1D, LSTM, GRU, Bidirectional, Dense,Flatten

2025-05-24 19:10:46.618457: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-05-24 19:10:46.618645: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-05-24 19:10:46.760523: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [ ]:
df=pd.read_csv("/kaggle/input/spam-text-message-classification/SPAM text message 20170820 - Data.csv")
df

,Category,Message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."
...,...,...
5567,spam,This is the 2nd time we have tried 2 contact u...
5568,ham,Will ü b going to esplanade fr home?
5569,ham,"Pity, * was in mood for that. So...any other s..."
5570,ham,The guy did some bitching but I acted like i'd...


In [ ]:
label=df["Category"].astype('category').cat.codes
label

0       0
1       0
2       1
3       0
4       0
       ..
5567    1
5568    0
5569    0
5570    0
5571    0
Length: 5572, dtype: int8

In [ ]:
test=df[4457:]
test_text=test["Message"]
test_label=test["Category"].astype('category').cat.codes
test_label

4457    0
4458    0
4459    0
4460    1
4461    0
       ..
5567    1
5568    0
5569    0
5570    0
5571    0
Length: 1115, dtype: int8

In [ ]:
tokenizer = Tokenizer(num_words=1000,oov_token="<OOV>")
tokenizer.fit_on_texts(df["Message"])
sequences = tokenizer.texts_to_sequences(df["Message"])
sequences = pad_sequences(sequences, padding='post')
len_sequences=len(sequences)
s=sequences.shape[1]

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(sequences, label, test_size=0.2, random_state=42)

In [ ]:
embedding_dim = 128
# Create the model
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(1000, embedding_dim, input_length=len_sequences),
    tf.keras.layers.Dense(128, activation='tanh'),
    tf.keras.layers.Dense(512, activation='tanh'),
    tf.keras.layers.GlobalAveragePooling1D(),
    tf.keras.layers.Dense(256, activation='tanh'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

# Compile the model
model.compile(loss='binary_crossentropy', optimizer=tf.keras.optimizers.Adam(learning_rate=0.001), metrics=['accuracy'])

# Train the model


model.summary()

Model: "sequential_5"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_5 (Embedding)         │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_14 (Dense)                │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_15 (Dense)                │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling1d_5      │ ?                      │   0 (unbuilt) │
│ (GlobalAveragePooling1D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_16 (Dense)                │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_17 (Dense)                │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

In [ ]:
history = model.fit(X_train, y_train, batch_size=128, epochs=50, validation_data=(X_test, y_test), callbacks=[tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)])

Epoch 1/50
35/35 ━━━━━━━━━━━━━━━━━━━━ 13s 307ms/step - accuracy: 0.7746 - loss: 0.4330 - val_accuracy: 0.8664 - val_loss: 0.3680
Epoch 2/50
35/35 ━━━━━━━━━━━━━━━━━━━━ 10s 288ms/step - accuracy: 0.8636 - loss: 0.3646 - val_accuracy: 0.8834 - val_loss: 0.3133
Epoch 3/50
35/35 ━━━━━━━━━━━━━━━━━━━━ 10s 287ms/step - accuracy: 0.8786 - loss: 0.2994 - val_accuracy: 0.8646 - val_loss: 0.3917
Epoch 4/50
35/35 ━━━━━━━━━━━━━━━━━━━━ 10s 290ms/step - accuracy: 0.8795 - loss: 0.4210 - val_accuracy: 0.8987 - val_loss: 0.2905
Epoch 5/50
35/35 ━━━━━━━━━━━━━━━━━━━━ 11s 303ms/step - accuracy: 0.9033 - loss: 0.2244 - val_accuracy: 0.9480 - val_loss: 0.1497
Epoch 6/50
35/35 ━━━━━━━━━━━━━━━━━━━━ 10s 290ms/step - accuracy: 0.9488 - loss: 0.1417 - val_accuracy: 0.9229 - val_loss: 0.2044
Epoch 7/50
35/35 ━━━━━━━━━━━━━━━━━━━━ 10s 289ms/step - accuracy: 0.8639 - loss: 0.5265 - val_accuracy: 0.9049 - val_loss: 0.3039
Epoch 8/50
35/35 ━━━━━━━━━━━━━━━━━━━━ 11s 303ms/step - accuracy: 0.9068 - loss: 0.2316 - val_accu

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.utils import shuffle

loss, acc = model.evaluate(X_test, y_test)
print(f"accuracy: {acc:.2f}")

y_pred = (model.predict(X_test) > 0.5).astype("int32")

print("\nreport:")
print(classification_report(y_test, y_pred, target_names=["Not Spam", "Spam"]))

35/35 ━━━━━━━━━━━━━━━━━━━━ 1s 21ms/step - accuracy: 0.9885 - loss: 0.0535
دقة النموذج: 0.99
35/35 ━━━━━━━━━━━━━━━━━━━━ 1s 22ms/step

تقرير التصنيف:
              precision    recall  f1-score   support

    Not Spam       0.99      1.00      0.99       966
        Spam       0.97      0.94      0.96       149

    accuracy                           0.99      1115
   macro avg       0.98      0.97      0.97      1115
weighted avg       0.99      0.99      0.99      1115

